<a href="https://colab.research.google.com/github/DinaZakaria/AFLW2000_HeadPose/blob/master/01_ML1_Project_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive
import PIL.Image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from skimage.io import imread, imsave
from glob import glob
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Functions

In [46]:
def get_image_XY (image, mat_file):
  XY_crop = []
  mat = sio.loadmat(mat_file)
  XY = mat['pt3d_68'][0:2]
  X_max = XY[0].max()
  Y_max = XY[1].max()
  X_min = XY[0].min()
  Y_min = XY[1].min()
  image = imread (image)
  croped =image[int(Y_min):int(Y_max), int(X_min):int(X_max)]
  X_crop = XY[0]-X_min
  Y_crop = XY[1]-Y_min
  XY_crop.append(X_crop)
  XY_crop.append(Y_crop)
  # for i in [30, 8, 36, 45, 48, 54]:
  #   XY_crop.append(X_crop[i])
  #   XY_crop.append(Y_crop[i])
  XY_crop = np.array(XY_crop).flatten()
  return XY_crop

In [47]:
def get_pyr(mat_file):
  #pitch, yaw, roll
  pyr = mat_file['Pose_Para'][:, 0:3][0]
  return pyr

In [48]:
def create_features(image_path_list, mat_path_list):
  features = list()
  for i in range (0,len(image_path_list)):
    XY = get_image_XY(image_path_list[i], mat_path_list[i])
    #print(XY)
    features.append(XY)
  return np.array(features)

In [49]:
def create_labels (mat_path_list):
  labels = list()
  for i in  range(0, len(mat_path_list)):
    mat_file = sio.loadmat(mat_path_list[i])
    pyr = get_pyr(mat_file)
    labels.append(pyr)
  return np.array(labels)

In [50]:
def create_df(image_path_list, mat_path_list):
  # data = np.array([])
  features = create_features(image_path_list, mat_path_list)
  labels = create_labels(mat_path_list)
  data = np.concatenate((features, labels), axis = 1)
  df = pd.DataFrame(data)
  return df

# Reading Dataset

In [51]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
Path = 'drive/My Drive/AI-Pro Track ITI/ML1/Project/DATA/aflw2000_3d_data'


In [53]:
image_path_list= []
image_path_list.extend(glob(os.path.join(Path, "AFLW2000", "*.jpg")))
mat_path_list = []
mat_path_list.extend(glob(os.path.join(Path, "AFLW2000", "*.mat")))

# for i, image_path in enumerate(image_path_list):
#     # read image
#     image = imread(image_path)
print(len(image_path_list))
print(len(mat_path_list))


2000
2000


# Dataframe

In [54]:
df = create_df(image_path_list, mat_path_list)
df

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,0.000000,0.981430,3.177963,7.520325,16.636810,36.479370,62.062347,88.868835,110.733765,118.650955,...,102.307434,104.603790,105.392700,106.522430,105.473969,104.190979,102.097076,-0.068547,-0.854021,0.218333
1,0.000000,2.627716,8.086754,14.034782,24.569130,43.002541,65.318535,94.591743,129.564789,159.050568,...,124.945862,123.593628,123.707397,127.578278,132.781006,133.528320,133.139709,0.027256,-0.247779,-0.010263
2,0.000000,2.811073,8.599014,14.774353,24.622337,42.561806,64.624336,88.097191,116.038246,139.069031,...,124.044159,123.918488,124.202271,127.209229,130.290009,131.033295,129.797485,-0.167856,-0.361028,0.058591
3,9.189560,13.021408,19.371765,24.665512,31.373764,41.456253,50.984695,61.400986,85.168381,112.612656,...,129.959900,131.272278,131.317719,133.843475,145.569031,147.044464,143.419983,-0.373085,0.293578,-0.125011
4,0.000000,5.628967,13.232834,21.585968,34.941544,56.668945,82.074783,110.237274,139.773102,160.022217,...,110.728516,109.649475,109.133148,110.911469,111.923309,113.186951,113.483612,0.036318,-0.447323,-0.115917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,21.561691,25.811447,32.422134,38.263092,43.471085,47.248810,47.666733,52.505356,69.010727,92.375259,...,101.394165,101.891266,101.157593,99.480896,106.234344,107.861328,106.594971,-0.218161,0.493541,-0.201171
1996,65.364685,68.758057,73.024994,74.687347,73.470276,67.888962,58.108994,49.526123,55.484039,73.191803,...,104.679474,99.255981,94.589630,91.891541,97.009796,101.801483,106.304535,0.236384,0.983607,-0.383734
1997,4.427643,9.238068,17.643219,24.182190,29.904846,36.717194,43.515259,57.460648,85.145340,116.014679,...,121.408234,120.166931,119.589172,121.766174,126.348846,127.412476,128.160187,0.004142,0.229272,-0.080178
1998,22.984161,36.283844,51.948059,65.758560,79.150574,91.497543,101.220642,112.787872,133.944183,155.966736,...,130.242004,125.732666,121.596771,112.713654,119.805725,124.505981,128.966248,-0.075998,0.314266,-0.553940


In [55]:
df.to_csv("AFLW200_CSV.csv", index = None, header = True)

In [56]:
X = df.iloc[:,0:136]
X

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,0.000000,0.981430,3.177963,7.520325,16.636810,36.479370,62.062347,88.868835,110.733765,118.650955,...,109.111389,105.080048,96.829559,102.307434,104.603790,105.392700,106.522430,105.473969,104.190979,102.097076
1,0.000000,2.627716,8.086754,14.034782,24.569130,43.002541,65.318535,94.591743,129.564789,159.050568,...,142.914368,139.086304,131.199585,124.945862,123.593628,123.707397,127.578278,132.781006,133.528320,133.139709
2,0.000000,2.811073,8.599014,14.774353,24.622337,42.561806,64.624336,88.097191,116.038246,139.069031,...,141.068115,135.634155,127.263855,124.044159,123.918488,124.202271,127.209229,130.290009,131.033295,129.797485
3,9.189560,13.021408,19.371765,24.665512,31.373764,41.456253,50.984695,61.400986,85.168381,112.612656,...,157.185822,145.055084,128.470276,129.959900,131.272278,131.317719,133.843475,145.569031,147.044464,143.419983
4,0.000000,5.628967,13.232834,21.585968,34.941544,56.668945,82.074783,110.237274,139.773102,160.022217,...,122.828003,120.564850,116.427887,110.728516,109.649475,109.133148,110.911469,111.923309,113.186951,113.483612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,21.561691,25.811447,32.422134,38.263092,43.471085,47.248810,47.666733,52.505356,69.010727,92.375259,...,115.640564,110.280731,101.560455,101.394165,101.891266,101.157593,99.480896,106.234344,107.861328,106.594971
1996,65.364685,68.758057,73.024994,74.687347,73.470276,67.888962,58.108994,49.526123,55.484039,73.191803,...,113.484253,117.182159,118.096100,104.679474,99.255981,94.589630,91.891541,97.009796,101.801483,106.304535
1997,4.427643,9.238068,17.643219,24.182190,29.904846,36.717194,43.515259,57.460648,85.145340,116.014679,...,138.018280,136.047943,128.165894,121.408234,120.166931,119.589172,121.766174,126.348846,127.412476,128.160187
1998,22.984161,36.283844,51.948059,65.758560,79.150574,91.497543,101.220642,112.787872,133.944183,155.966736,...,137.387329,140.728149,140.594513,130.242004,125.732666,121.596771,112.713654,119.805725,124.505981,128.966248


In [57]:
df_new = pd.read_csv("AFLW200_CSV.csv", header = None)

In [27]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,129.000000,130.000000,131.000000,132.000000,133.000000,134.000000,135.000000,136.000000,137.000000,138.000000
1,0.000000,0.981430,3.177963,7.520325,16.636810,36.479370,62.062347,88.868835,110.733765,118.650955,...,102.307434,104.603790,105.392700,106.522430,105.473969,104.190979,102.097076,-0.068547,-0.854021,0.218333
2,0.000000,2.627716,8.086754,14.034782,24.569130,43.002541,65.318535,94.591743,129.564789,159.050568,...,124.945862,123.593628,123.707397,127.578278,132.781006,133.528320,133.139709,0.027256,-0.247779,-0.010263
3,0.000000,2.811073,8.599014,14.774353,24.622337,42.561806,64.624336,88.097191,116.038246,139.069031,...,124.044159,123.918488,124.202271,127.209229,130.290009,131.033295,129.797485,-0.167856,-0.361028,0.058591
4,9.189560,13.021408,19.371765,24.665512,31.373764,41.456253,50.984695,61.400986,85.168381,112.612656,...,129.959900,131.272278,131.317719,133.843475,145.569031,147.044464,143.419983,-0.373085,0.293578,-0.125011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,21.561691,25.811447,32.422134,38.263092,43.471085,47.248810,47.666733,52.505356,69.010727,92.375259,...,101.394165,101.891266,101.157593,99.480896,106.234344,107.861328,106.594971,-0.218161,0.493541,-0.201171
1997,65.364685,68.758057,73.024994,74.687347,73.470276,67.888962,58.108994,49.526123,55.484039,73.191803,...,104.679474,99.255981,94.589630,91.891541,97.009796,101.801483,106.304535,0.236384,0.983607,-0.383734
1998,4.427643,9.238068,17.643219,24.182190,29.904846,36.717194,43.515259,57.460648,85.145340,116.014679,...,121.408234,120.166931,119.589172,121.766174,126.348846,127.412476,128.160187,0.004142,0.229272,-0.080178
1999,22.984161,36.283844,51.948059,65.758560,79.150574,91.497543,101.220642,112.787872,133.944183,155.966736,...,130.242004,125.732666,121.596771,112.713654,119.805725,124.505981,128.966248,-0.075998,0.314266,-0.553940


In [30]:
X = df.iloc[:,0:136].to_numpy()
X

array([[  0.        ,   0.98143005,   3.17796326, ..., 105.47396851,
        104.190979  , 102.09707642],
       [  0.        ,   2.62771606,   8.08675385, ..., 132.78100586,
        133.52832031, 133.13970947],
       [  0.        ,   2.8110733 ,   8.59901428, ..., 130.29000854,
        131.03329468, 129.79748535],
       ...,
       [  4.42764282,   9.23806763,  17.64321899, ..., 126.34884644,
        127.41247559, 128.16018677],
       [ 22.98416138,  36.28384399,  51.94805908, ..., 119.8057251 ,
        124.50598145, 128.96624756],
       [  3.04327393,  10.9591217 ,  20.03878784, ..., 118.12960815,
        121.75390625, 123.72192383]])

In [34]:
p = df_new[[136]].to_numpy()
p


array([[ 1.36000000e+02],
       [-6.85465112e-02],
       [ 2.72560306e-02],
       ...,
       [ 4.14238498e-03],
       [-7.59984627e-02],
       [-1.40524477e-01]])